In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Dataset path (change if needed)
dataset_path = r"E:\Soil Quality\Orignal-Dataset"

# Data generators (80% train, 20% validation)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=True
)

# CNN Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train Model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save model
model.save("soil_cnn.h5")
print("✅ Model saved as soil_cnn.h5")


Found 954 images belonging to 7 classes.
Found 234 images belonging to 7 classes.


e:\Soil Quality\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
e:\Soil Quality\myenv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 242ms/step - accuracy: 0.3962 - loss: 1.7411 - val_accuracy: 0.5598 - val_loss: 1.1667
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 212ms/step - accuracy: 0.6038 - loss: 0.9985 - val_accuracy: 0.5855 - val_loss: 0.9897
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 213ms/step - accuracy: 0.7075 - loss: 0.7987 - val_accuracy: 0.6496 - val_loss: 0.9157
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 213ms/step - accuracy: 0.7296 - loss: 0.7211 - val_accuracy: 0.7222 - val_loss: 0.7732
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 253ms/step - accuracy: 0.7746 - loss: 0.5968 - val_accuracy: 0.6838 - val_loss: 0.8997
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 260ms/step - accuracy: 0.8208 - loss: 0.5009 - val_accuracy: 0.7607 - val_loss: 0.7080
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 251ms/step - accuracy: 0.7925 - loss: 0.5256 - val_accuracy: 0.6795 - val_loss: 0.9341
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 250ms/step - accuracy: 0.8197 - loss: 0.4987 - val_accuracy: 0.

✅ Model saved as soil_cnn.h5
